<a href="https://colab.research.google.com/github/luisArthurRodriguesDaSilva/estudo-dados-eleicoes/blob/master/arquivos/reunindoOsDados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 | importações de bibliotecas e recursos

In [ ]:
import pandas as pd
import numpy as np
import requests
import pprint
from matplotlib import pyplot as plt
import seaborn as sns
import cmath
import threading
import time
import json
import statistics

# 2 | funções para fazer as devidas requisições na api

In [ ]:
def getDeputadoEstadualUrl(uf):
  return f'https://resultados.tse.jus.br/oficial/ele2022/546/dados-simplificados/{uf}/{uf}-c0007-e000546-r.json'

def getPresidentUrl(uf):
  return f'https://resultados.tse.jus.br/oficial/ele2022/544/dados-simplificados/{uf}/{uf}-c0001-e000544-r.json'

def getGovernadorUrl(uf):
  return f'https://resultados.tse.jus.br/oficial/ele2022/546/dados-simplificados/{uf}/{uf}-c0003-e000546-r.json'

def getDeputadoFederalUrl(uf):
  return f'https://resultados.tse.jus.br/oficial/ele2022/546/dados-simplificados/{uf}/{uf}-c0006-e000546-r.json'

def getSenadorUrl(uf):
  return f'https://resultados.tse.jus.br/oficial/ele2022/546/dados-simplificados/{uf}/{uf}-c0005-e000546-r.json'

gets= [getPresidentUrl, getSenadorUrl, getDeputadoFederalUrl, getDeputadoEstadualUrl, getGovernadorUrl]
categoryKeys= ['Presidente', 'Senador', 'DeputadoFederal', 'DeputadoEstadual', 'Governador']
brasil_ufs = ['ac','al','am','ap','ba','ce','df','es','go','ma','mt','ms','mg','pa','pb','pr','pe','pi','rj','rn','ro','rs','rr','sc','se','sp','to']

LULA_TAG          = 'LULA' 
BOLSONARO_TAG     = 'JAIR BOLSONARO' 
CIRO_TAG          = 'CIRO GOMES' 
TEBET_TAG         = 'SIMONE TEBET' 
SORAYA_TAG        = 'SORAYA THRONICKE' 
DAVILA_TAG        =  "FELIPE D'AVILA"
PADRE_TAG         = 'PADRE KELMON' 
SOFIA_MANZANO_TAG = 'SOFIA MANZANO' 
LEO_PERICLES_TAG  ='LÉO PÉRICLES'
VERA_TAG          = 'VERA' 
EYMAEL_TAG        = 'CONSTITUINTE EYMAEL'
candidatesTags    = [LULA_TAG, BOLSONARO_TAG, CIRO_TAG, TEBET_TAG, SORAYA_TAG, DAVILA_TAG, PADRE_TAG, SOFIA_MANZANO_TAG, LEO_PERICLES_TAG, VERA_TAG, EYMAEL_TAG]
candidatesColors   =['red', 'green', 'blue','brown', 'green', 'blue','red', 'green', 'blue','green','yellow']
politicalColors = {}
for i, tag in enumerate(candidatesTags):
  politicalColors[tag] = candidatesColors[i]

# 3 | Funções para a coleta de **dados**

In [ ]:
arq =r'/content/drive/MyDrive/completeUfsNames.json'
with open(arq,'r') as f:
  completeUfsNames = json.load(f)[0]

In [ ]:
def getPopulationByUf(uf,info = 'População (2021)'):
  url = 'https://pt.wikipedia.org/wiki/Lista_de_unidades_federativas_do_Brasil_por_popula%C3%A7%C3%A3o'
  table = pd.read_html(url , match='Posição')
  ufs_population_data = table[0]
  region_population_data = table[1]
  ufName = completeUfsNames[uf.upper()]
  keysArray = list(ufs_population_data.keys()[0:].array)
  positionOfInfo = keysArray.index(info)
  usedData = ufs_population_data.values
  try:
    filteredUsedData = list(filter( lambda x: x[1] == ufName, usedData))[0]
  except Exception as e:
    return 'não encontrado'
  requiredInfo = str(filteredUsedData[positionOfInfo])
  return int(requiredInfo.replace(u'\xa0', ''))

def getPibByUf(uf, info= 'PIB em 2019 (1.000.000 R$)'):
  url = 'https://www.ibge.gov.br/explica/pib.php'
  table = pd.read_html(url , match='Unidades da Federação')
  pibsData = table[0]
  ufName = completeUfsNames[uf.upper()]
  try:
    filteredUsedData = list(filter(lambda x: x[0] == ufName ,pibsData.values))[0][1]
  except Exception as e:
    return 'não encontrado'
  requiredInfo = str(filteredUsedData.replace('.',''))
  return int(requiredInfo)

def getPibPerCapta(uf):
  return float(abs(getPibByUf(uf) / getPopulationByUf(uf)))

In [ ]:
def takeData(url):
  re=requests.get(url).json()
  candidatos = re['cand']

  df=pd.DataFrame(candidatos)
  df=df[['nm','n','dvt','vap','pvap']]
  df.rename(columns={'nm':'Nome do Candidato',
                  'n':'Numero do Candidato',
                  'dvt':'Validade',
                  'vap':'Votos',
                  'pvap':'(%) Votos'}, inplace=True)
  
  df['(%) Votos']=df['(%) Votos'].str.replace(',','.')
  df['(%) Votos']=df['(%) Votos'].astype(float)
  df['Votos']=df['Votos'].astype(int)
  df['Nome do Candidato']=df['Nome do Candidato'].str.replace('D&apos;',"D'")
  df=df.sort_values(by='Votos', ascending=False)
  return(df)

In [ ]:
def getAllDataByUf(uf):
  info = {}
  for index, category in enumerate(categoryKeys):
    try:
      info[category] = takeData(gets[index](uf))
      info['populacao'] = getPopulationByUf(uf) #deve sair do loop pra n ser sobrescrito varias vezes
      info['pib'] = getPibByUf(uf)
      info['pib per-capta'] = getPibPerCapta(uf)
    except:
      if uf == 'df':
        info[category] = 'df não tem prefeito'
      else:
        info[category] = 'this is brasil'
  return info

# 4 | Organização dos dados através do electionsData
### ele é um objeto mais organizado contendo todas as informações separadas por estado

In [ ]:
electionsData = {}
electionsData['br'] = getAllDataByUf('br')

def atribute(uf):
  electionsData[uf] = getAllDataByUf(uf)

for i, uf in  enumerate(brasil_ufs):
  threading.Thread(target=atribute, args=(uf,)).start()
time.sleep(5)